# 📋 Jupyter Notebook Quick Shortcuts (Markdown cell)
# Press 'H' in command mode to view all shortcuts
# 
# **Command Mode** (press Esc to enter):
# - A: Insert cell above
# - B: Insert cell below
# - D D: Delete current cell
# - Z: Undo delete
# - M: Change cell to Markdown
# - Y: Change cell to Code
# - Shift + Enter: Run cell and go to next
# - Ctrl + Enter: Run cell (stay on it)
#
# **Edit Mode** (press Enter to enter):
# - Tab: Auto-complete
# - Shift + Tab: Show function hint
# - Ctrl + /: Comment/uncomment line
